In [1]:
import torch
import torchvision
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear, CrossEntropyLoss

In [2]:
# 使用Sequential来优化网络结构
class MyModule(torch.nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()

        self.model = torch.nn.Sequential(
            Conv2d(3, 32, 5, padding=2),  # Input: 3@32×32, Output: 32@32×32
            MaxPool2d(2),  # Input: 32@32×32, Output: 32@16×16
            Conv2d(32, 32, 5, padding=2),  # Input: 32@16×16, Output: 32@16×16
            MaxPool2d(2),  # Input: 32@16×16, Output: 32@8×8
            Conv2d(32, 64, 5, padding=2),  # Input: 32@8×8, Output: 64@8×8
            MaxPool2d(2),  # Input: 64@8×8, Output: 64@4×4
            Flatten(),  # Input: 64@4×4, Output: 1024
            Linear(1024, 64),  # Input: 1024, Output: 64
            Linear(64, 10)  # Input: 64, Output: 10
        )

    def forward(self, inputs):
        outputs = self.model(inputs)
        return outputs


module = MyModule()
print(module)

MyModule(
  (model): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=1024, out_features=64, bias=True)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)


In [3]:
transformer = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])

DATA_DIR = '../05-Transforms/data/'

test_data = torchvision.datasets.CIFAR10(
    root=DATA_DIR,
    transform=transformer,
    train=False,
    download=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_data,
    batch_size=1,
    shuffle=True,
    num_workers=4,
    drop_last=True
)

Files already downloaded and verified


In [4]:
loss = CrossEntropyLoss()
optim = torch.optim.SGD(module.parameters(), lr=0.01)

In [5]:
for i, data in enumerate(test_loader):
    imgs, targets = data
    outputs = module(imgs)
    res_loss = loss(outputs, targets)
    optim.zero_grad()
    res_loss.backward()
    optim.step()
    print(res_loss)

# 只经过一轮遍历学习的话，感觉loss也并没有很明显的下降，感觉用的是随机梯度下降？？？

tensor(2.2282, grad_fn=<NllLossBackward0>)
tensor(2.3551, grad_fn=<NllLossBackward0>)
tensor(2.3727, grad_fn=<NllLossBackward0>)
tensor(2.3406, grad_fn=<NllLossBackward0>)
tensor(2.2785, grad_fn=<NllLossBackward0>)
tensor(2.2657, grad_fn=<NllLossBackward0>)
tensor(2.2047, grad_fn=<NllLossBackward0>)
tensor(2.2966, grad_fn=<NllLossBackward0>)
tensor(2.3316, grad_fn=<NllLossBackward0>)
tensor(2.2806, grad_fn=<NllLossBackward0>)
tensor(2.4270, grad_fn=<NllLossBackward0>)
tensor(2.3383, grad_fn=<NllLossBackward0>)
tensor(2.3497, grad_fn=<NllLossBackward0>)
tensor(2.3925, grad_fn=<NllLossBackward0>)
tensor(2.3079, grad_fn=<NllLossBackward0>)
tensor(2.2811, grad_fn=<NllLossBackward0>)
tensor(2.2201, grad_fn=<NllLossBackward0>)
tensor(2.4867, grad_fn=<NllLossBackward0>)
tensor(2.4273, grad_fn=<NllLossBackward0>)
tensor(2.3692, grad_fn=<NllLossBackward0>)
tensor(2.3408, grad_fn=<NllLossBackward0>)
tensor(2.2877, grad_fn=<NllLossBackward0>)
tensor(2.2661, grad_fn=<NllLossBackward0>)
tensor(2.29

In [6]:
loss = CrossEntropyLoss()
optim = torch.optim.SGD(module.parameters(), lr=0.0001)

In [7]:
for epoch in range(20):
    epoch_loss = 0  # 对每一轮的loss进行累加，观察经过多轮学习后，loss是否有下降
    for i, data in enumerate(test_loader):
        imgs, targets = data
        outputs = module(imgs)
        res_loss = loss(outputs, targets)
        optim.zero_grad()
        res_loss.backward()
        optim.step()
        epoch_loss += res_loss
        print(f'第{epoch}轮中的第{i}批数据训练Loss值为：{res_loss.item()}.')

    print(f'第{epoch}轮的累积Loss值为：{epoch_loss.item()}.')

# 如果，每轮循环下来loss不降反增，尝试减小学习率

tensor(14272.0645, grad_fn=<AddBackward0>)
tensor(13447.1875, grad_fn=<AddBackward0>)
tensor(13045.7441, grad_fn=<AddBackward0>)
tensor(12753.6035, grad_fn=<AddBackward0>)
tensor(12508.4531, grad_fn=<AddBackward0>)
tensor(12302.8574, grad_fn=<AddBackward0>)
tensor(12116.5771, grad_fn=<AddBackward0>)
tensor(11941.4707, grad_fn=<AddBackward0>)
tensor(11775.6465, grad_fn=<AddBackward0>)
tensor(11619.3770, grad_fn=<AddBackward0>)
tensor(11468.9473, grad_fn=<AddBackward0>)
tensor(11316.9297, grad_fn=<AddBackward0>)
tensor(11182.4756, grad_fn=<AddBackward0>)
tensor(11041.6973, grad_fn=<AddBackward0>)
tensor(10911.4316, grad_fn=<AddBackward0>)
tensor(10770.9912, grad_fn=<AddBackward0>)
tensor(10631.3779, grad_fn=<AddBackward0>)
tensor(10504.6367, grad_fn=<AddBackward0>)
tensor(10368.0273, grad_fn=<AddBackward0>)
tensor(10239.2461, grad_fn=<AddBackward0>)


In [10]:
# item()的作用——取出tensor中的值
torch.tensor(5).item()

5

In [11]:
# 当Tensor中只有一个元素时，才可以使用tensor.item()
torch.tensor([2, 3]).item()

RuntimeError: a Tensor with 2 elements cannot be converted to Scalar